```{contents}
```
## Structured Tools

### What Structured Tools Are

**Structured Tools** are tools with **explicit, strongly typed input schemas**, typically defined using **Pydantic models**.
They ensure that when an LLM calls a tool, the inputs are **validated, predictable, and safe**.

> Structured Tools = **Tools with guaranteed input structure**

They are critical for **production-grade agents**.

---

### Why Structured Tools Exist

Free-form or loosely typed tools can:

* Receive malformed inputs
* Cause runtime errors
* Be exploited by prompt injection
* Produce inconsistent behavior

Structured tools enforce:

* Input validation
* Type safety
* Clear contracts
* Safer tool execution

---

### Structured Tool vs Regular Tool

| Aspect                 | Regular Tool (`@tool`) | Structured Tool     |
| ---------------------- | ---------------------- | ------------------- |
| Input typing           | Inferred               | Explicit (Pydantic) |
| Validation             | Basic                  | Strict              |
| Production safety      | Medium                 | High                |
| Complex inputs         | Hard                   | Easy                |
| Recommended for writes | ❌                      | ✅                   |

---

### Where Structured Tools Fit

```
LLM / Agent
   ↓
Structured Tool (validated input)
   ↓
Deterministic execution
   ↓
Tool Result
```

---

### Core Components of a Structured Tool

1. **Tool name**
2. **Tool description**
3. **Pydantic input schema**
4. **Callable function**
5. **Validated output**

---

### Demonstration: Structured Tool (End-to-End)

#### Step 1: Define Input Schema



In [1]:
from pydantic import BaseModel, Field

class TicketQuery(BaseModel):
    source: str = Field(..., description="Ticket system name, e.g., jira")
    status: str = Field(..., description="Ticket status: open, closed")




This schema:

* Enforces required fields
* Adds semantic descriptions
* Prevents invalid inputs

---

#### Step 2: Define Tool Logic



In [ ]:
def ticket_lookup(source: str, status: str) -> int:
    if source == "jira" and status == "open":
        return 128
    return 0


---

#### Step 3: Create the Structured Tool



In [3]:
from langchain_classic.tools import StructuredTool

ticket_tool = StructuredTool.from_function(
    name="ticket_lookup",
    description="Return number of tickets by source and status",
    func=ticket_lookup,
    args_schema=TicketQuery
)




---

### How the LLM Sees the Tool (Conceptual)

```json
{
  "name": "ticket_lookup",
  "description": "Return number of tickets by source and status",
  "parameters": {
    "source": { "type": "string" },
    "status": { "type": "string" }
  }
}
```

The LLM **cannot pass invalid fields**.

---

### Using Structured Tools with an Agent

#### Create Agent



In [5]:
from langchain_classic.agents import create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chat_models import ChatOpenAI
prompt = ChatPromptTemplate.from_messages([
    ("system", "Use tools when needed."),
    ("human", "{input}"),
    ("assistant", "{agent_scratchpad}")
])

llm = ChatOpenAI(temperature=0)
agent = create_openai_tools_agent(
    llm=llm,
    tools=[ticket_tool],
    prompt=prompt
)


C:\Users\sangouda\AppData\Local\Temp\ipykernel_28076\1468259600.py:10: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the `langchain-openai package and should be used instead. To use it run `pip install -U `langchain-openai` and import as `from `langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0)



---

#### Execute

In [9]:
from langchain_classic.agents import AgentExecutor

executor = AgentExecutor(
    agent=agent,
    tools=[ticket_tool],
    max_iterations=3,
)

answer = executor.invoke({
    "input": "How many open Jira tickets are there?"
})

answer["output"]

'There are 128 open Jira tickets.'



---

### What Happens Internally

1. Agent decides tool is required
2. Produces structured arguments:

   ```json
   { "source": "jira", "status": "open" }
   ```
3. LangChain validates input against `TicketQuery`
4. Function executes
5. Result returned to agent
6. Final answer generated

---

### Validation Failure Example (Safety)

If the model tries:

```json
{ "system": "jira", "state": "open" }
```

LangChain:

* ❌ Rejects input
* ❌ Tool not executed
* ❌ No side effects

---

### Structured Tools for Write Operations (IMPORTANT)

For actions like:

* Creating tickets
* Sending notifications
* Updating records

**Always use Structured Tools**.

Example:

```python
class NotifyInput(BaseModel):
    message: str
    priority: str
```

---

### Structured Tools vs MCP

| Aspect     | Structured Tools | MCP            |
| ---------- | ---------------- | -------------- |
| Validation | Pydantic         | Contract-based |
| Scope      | In-process       | Cross-process  |
| Isolation  | Low              | High           |
| Enterprise | Medium           | Very High      |

---

### Best Practices

* Use Structured Tools for all write operations
* Keep schemas minimal
* Use enums for constrained values
* Add clear descriptions
* Log validation errors
* Separate read and write tools

---

### Common Mistakes

* Overloading a single tool
* Weak descriptions
* No input validation
* Using regular tools for destructive actions

---

### When to Use Structured Tools

* Production systems
* APIs and DB writes
* Enterprise workflows
* Compliance-sensitive actions

---

### When Regular Tools Are Enough

* Simple reads
* Prototyping
* Experiments

---

### Interview-Ready Summary

> “Structured tools are tools with explicit input schemas that enforce strict validation and predictable behavior. They are essential for production-grade agents, especially when tools have side effects.”

---

### Rule of Thumb

* **Reads → Regular tools**
* **Writes → Structured tools**
* **Enterprise → Structured tools + guardrails**

